In [ ]:
import os
caffe_root = '../caffe-portraitseg/'
import sys
sys.path.insert(0, caffe_root + 'python')

In [ ]:
import numpy as np
from matplotlib import pyplot as plt
import caffe
caffe.set_mode_cpu()

In [ ]:
MODEL_FILE = './our_models/deploy_3channels.prototxt'
PRETRAINED = './our_models/bgr.caffemodel'

In [ ]:
net = caffe.Net(MODEL_FILE, PRETRAINED, caffe.TEST)

In [ ]:
net.blobs['data'].reshape(1, 3, 800, 600)

In [ ]:
# create transformer for the input called 'data'
transformer = caffe.io.Transformer({'data': net.blobs['data'].data.shape})

transformer.set_transpose('data', (2,0,1))  # move image channels to outermost dimension
transformer.set_mean('data', np.asarray([122.675, 116.669, 114.008]))            # subtract the dataset-mean value in each channel
transformer.set_raw_scale('data', 255)      # rescale from [0, 1] to [0, 255]
transformer.set_channel_swap('data', (2,1,0))  # swap channels from RGB to BGR

In [ ]:
image = caffe.io.load_image('00321.jpg')

In [ ]:
transformed_image = transformer.preprocess('data', image)
transformed_image /= 255

In [ ]:
net.blobs['data'].data[...] = transformed_image

In [ ]:
output = net.forward() 

In [ ]:
res = output['upscore']

In [ ]:
res = res.reshape(res.shape[1:])

In [ ]:
diffs = np.exp(res[1,:,:] - res[0,:,:])

In [ ]:
finalres = diffs/(1+diffs)

---

In [1]:
import scipy.io as scio

In [2]:
testlistPath = '../data/testlist.mat'
outputPath = './Output_PortraitFCN/'

In [3]:
testlist = scio.loadmat(testlistPath)['testlist']

In [ ]:
for i in range(len(testlist)):
    image = scio.loadmat('../data/portraitFCN_data/%05d.mat' % testlist[0][i])['img']
    image = image.transpose((2,0,1))
    net.blobs['data'].data[...] = image
    output = net.forward() 
    res = output['upscore']
    res = res.reshape(res.shape[1:])
    scio.savemat(outputPath + '%05d_output.mat'% testlist[0][i], {'res':res})

In [12]:
image = scio.loadmat('../data/portraitFCN_data/%05d.mat' % testlist[0][0])['img']

In [13]:
image.shape

(800, 600, 3)

In [15]:
image.transpose((2,0,1))

array([[[-0.1961098 , -0.20003137, -0.19218824, ..., -0.11767843,
         -0.11375686, -0.11375686],
        [-0.1961098 , -0.1961098 , -0.19218824, ..., -0.11767843,
         -0.11375686, -0.11375686],
        [-0.18826667, -0.19218824, -0.19218824, ..., -0.11375686,
         -0.11375686, -0.11375686],
        ..., 
        [ 0.2038902 ,  0.2038902 ,  0.19996863, ...,  0.23134118,
          0.23134118,  0.23134118],
        [ 0.2038902 ,  0.19996863,  0.19996863, ...,  0.23134118,
          0.23134118,  0.23134118],
        [ 0.19996863,  0.19996863,  0.19996863, ...,  0.23134118,
          0.23134118,  0.23134118]],

       [[-0.21046667, -0.21438824, -0.21438824, ..., -0.18301569,
         -0.17517255, -0.17517255],
        [-0.21046667, -0.21046667, -0.21438824, ..., -0.18301569,
         -0.17517255, -0.17517255],
        [-0.20262353, -0.2065451 , -0.2065451 , ..., -0.17909412,
         -0.17517255, -0.17517255],
        ..., 
        [ 0.13855294,  0.13855294,  0.13463137, ...,